In [55]:
import pymc3 as pm
import arviz as az
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import warnings
from theano import shared,tensor
import time 
from datetime import datetime
from sklearn.metrics import mean_squared_error
import pickle

In [56]:
warnings.filterwarnings("ignore")

In [57]:
input_file='/Users/pryang/Downloads/EPA_OD_202305.csv'
df=pd.read_csv(input_file)
df=copy.deepcopy(df)
df.head()

,SiteName,County,AQI,Pollutant,Status,SO2,CO,CO_8hr,O3,O3_8hr,...,NO2,NOx,NO,WindSpeed,WindDirec,PublishTime,SO2_AVG,Longitude,Latitude,SiteId
0,豐原,臺中市,68.0,細懸浮微粒,普通,1.4,0.27,0.2,22.0,32.3,...,5.6,6.2,0.6,1.6,6,2023-05-01 00:00:00,1.0,120.742524,24.256997,28
1,西屯,臺中市,76.0,細懸浮微粒,普通,1.2,0.26,0.2,33.0,38.8,...,11.0,10.8,0.0,2.8,17,2023-05-01 00:00:00,1.0,120.616917,24.162197,32
2,大里,臺中市,61.0,細懸浮微粒,普通,1.4,0.33,0.3,24.8,32.8,...,9.9,10.6,0.6,1.7,342,2023-05-01 00:00:00,1.0,120.678444,24.099611,30
3,淡水,新北市,55.0,細懸浮微粒,普通,1.2,0.39,0.3,26.4,38.8,...,10.4,11.9,1.5,NaN,NaN,2023-05-01 00:00:00,1.0,121.449239,25.164500,10
4,忠明,臺中市,64.0,細懸浮微粒,普通,1.5,0.28,0.3,32.9,39.0,...,NaN,NaN,NaN,1.3,0,2023-05-01 00:00:00,1.0,120.641092,24.151958,31


In [58]:
Site=list(df.SiteId.unique())
Site.sort()
print("Unique SiteId:",Site)

Unique SiteId: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 75, 77, 78, 80, 83, 84]


In [59]:
df.PublishTime[0][:5]

'2023-'

In [60]:
time_start=input("Start Date (ex:05-01):")
time_end=input("End Date (ex:05-06):")
time_start=df.PublishTime[0][:5]+time_start+" 00:00:00"
time_end=df.PublishTime[0][:5]+time_end+" 00:00:00"
test1_id=int(input("First site to test:"))
test2_id=int(input("Second site to test:"))
df1=df[(df.SiteId==test1_id) & (df.PublishTime>=time_start) & (df.PublishTime<=time_end)]
df2=df[(df.SiteId==test2_id) & (df.PublishTime>=time_start) & (df.PublishTime<=time_end)]

In [61]:
print("For first data:")
features=['PM10','PM2.5_AVG','PM2.5','SiteId']
for feature in features:
    print(f"{feature} has nan value: ",df1[feature].isna().any())
    # print(df[df[feature].isna()].index)

For first data:
PM10 has nan value:  False
PM2.5_AVG has nan value:  False
PM2.5 has nan value:  False
SiteId has nan value:  False


In [62]:
print("For second data:")
features=['PM10','PM2.5_AVG','PM2.5','SiteId']
for feature in features:
    print(f"{feature} has nan value: ",df1[feature].isna().any())
    # print(df[df[feature].isna()].index)

For second data:
PM10 has nan value:  False
PM2.5_AVG has nan value:  False
PM2.5 has nan value:  False
SiteId has nan value:  False


In [63]:
def preprosessing(data):
    for feature in features:
        losts=data[data[feature].isna()].index
        for lost in losts:
            siteid = data.loc[lost, 'SiteId']
            s = data[data['SiteId'] == siteid][feature]
            data.loc[data['SiteId'] == siteid, feature] = s.fillna(method='ffill')
    return data
df1=preprosessing(df1)
df2=preprosessing(df2)

In [64]:
def standardize(data):
    for feature in features[:-2]:
        data[feature] =( data[feature] - data[feature].mean() ) / data[feature].std()
    return data
df1=standardize(df1)
df2=standardize(df2)

In [65]:
ref={}
for i in range(1,9):
    filename="dict"+str(i)+".pkl"
    with open(filename,"rb") as file:
        ref.update(pickle.load(file))

In [66]:
def calculate_result(row):
    site=int(row['SiteId'])
    a=ref[site]['alpha'].mean()
    b0=ref[site]['beta'][0].mean()
    b1=ref[site]['beta'][1].mean()
    result = a + b0 * row[features[0]]+  b1 * row[features[1]]
    return result

In [67]:
print("For first data:")
df1['result'] = df1.apply(calculate_result, axis=1)
mse = mean_squared_error(df1['PM2.5'].values, df1['result'].values)
print("Mean Squre Error:",mse)

For first data:
Mean Squre Error: 57.05887423217341


In [68]:
print("For second data:")
df2['result'] = df2.apply(calculate_result, axis=1)
mse = mean_squared_error(df2['PM2.5'].values, df2['result'].values)
print("Mean Squre Error:",mse)

For second data:
Mean Squre Error: 67.3679427815525
